In [1]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt


In [2]:
tickers = ['AAPL', 'C', 'AMZN', 'DIS', 'NVDA', 'UNH', 'MSFT']
etfs = ['SPY', 'IWM', 'DIA']

date_end = dt.datetime.now()
date_start = date_end - dt.timedelta(int(10*365))

port_data = yf.download(tickers + etfs, period = '10y')['Adj Close']

risk_data = yf.download(etfs, period = '10y')['Adj Close']

p_returns = port_data.pct_change()

ann_ret = p_returns[-252:]

ann_vol = (p_returns[-63:].std()) * (252 ** (1/2))

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  3 of 3 completed


In [3]:
port_data.head()

,AAPL,AMZN,C,DIA,DIS,IWM,MSFT,NVDA,SPY,UNH
Date,,,,,,,,,,
2013-10-07,15.203009,15.5015,38.710697,120.517563,58.718304,92.479622,27.885908,3.657556,139.549774,61.915363
2013-10-08,14.990747,15.1615,38.300938,119.328583,58.181942,90.984528,27.643057,3.627057,137.924438,61.323410
2013-10-09,15.166852,14.9115,38.525906,119.530838,57.809216,90.617294,27.693314,3.570750,138.024551,61.237576
2013-10-10,15.261921,15.2585,39.586468,122.102905,59.618309,92.803131,28.271112,3.610634,141.000046,63.468151
2013-10-11,15.360731,15.5445,39.546314,123.081665,60.191029,94.149666,28.580967,3.580134,141.908539,63.716957


In [4]:
result = pd.DataFrame(index = tickers)

result['weight'] = 100 / len(tickers)

result['ann_vol'] = ann_vol

result

,weight,ann_vol
AAPL,14.285714,0.215355
C,14.285714,0.196506
AMZN,14.285714,0.328352
DIS,14.285714,0.244820
NVDA,14.285714,0.398036
UNH,14.285714,0.224035
MSFT,14.285714,0.227751


In [5]:
port_data.cov()

,AAPL,AMZN,C,DIA,DIS,IWM,MSFT,NVDA,SPY,UNH
AAPL,3008.290641,2417.412633,154.312759,3778.563161,784.156457,1815.498841,5327.044033,5284.845741,5121.651408,7734.523221
AMZN,2417.412633,2660.039436,261.935261,3431.523183,1144.449587,1754.388839,4583.618896,3994.353064,4518.206720,6349.370717
C,154.312759,261.935261,93.753785,378.877894,203.534258,238.134118,346.180952,234.070714,438.762682,510.505734
DIA,3778.563161,3431.523183,378.877894,5347.125034,1329.123911,2621.509704,6958.613274,6551.755683,6957.111569,10375.034388
DIS,784.156457,1144.449587,203.534258,1329.123911,852.286045,818.491970,1593.642910,994.358796,1721.144157,1982.960067
IWM,1815.498841,1754.388839,238.134118,2621.509704,818.491970,1417.399703,3345.863297,3101.364035,3434.444957,4874.847974
MSFT,5327.044033,4583.618896,346.180952,6958.613274,1593.642910,3345.863297,9716.426343,9452.091100,9344.787560,13864.832944
NVDA,5284.845741,3994.353064,234.070714,6551.755683,994.358796,3101.364035,9452.091100,10838.545361,8975.398837,13227.846986
SPY,5121.651408,4518.206720,438.762682,6957.111569,1721.144157,3434.444957,9344.787560,8975.398837,9224.968805,13637.346067
UNH,7734.523221,6349.370717,510.505734,10375.034388,1982.960067,4874.847974,13864.832944,13227.846986,13637.346067,21595.236622


In [6]:
# result['beta_spy'] = ann_ret.cov()['SPY'] / ann_ret['SPY'].var()
# result['beta_iwm'] = ann_ret.cov()['IWM'] / ann_ret['IWM'].var()
# result['beta_dia'] = ann_ret.cov()['DIA'] / ann_ret['DIA'].var()

In [7]:
for etf in etfs:
    result['beta_' +etf.lower()] = ann_ret.cov()[etf] / ann_ret[etf].var()

In [8]:
result

,weight,ann_vol,beta_spy,beta_iwm,beta_dia
AAPL,14.285714,0.215355,1.327641,0.833916,1.345549
C,14.285714,0.196506,1.060960,0.921424,1.288149
AMZN,14.285714,0.328352,1.554363,0.994008,1.358296
DIS,14.285714,0.244820,1.229410,0.954847,1.407096
NVDA,14.285714,0.398036,2.151439,1.300109,1.881499
UNH,14.285714,0.224035,0.389200,0.221559,0.737294
MSFT,14.285714,0.227751,1.380857,0.759801,1.325091


In [9]:
drawdown = (port_data / port_data.rolling(window=252).max()) - 1

In [10]:
result['avg drawdown'] = drawdown.mean()
result['max_drawdown'] = drawdown.min()

result

,weight,ann_vol,beta_spy,beta_iwm,beta_dia,avg drawdown,max_drawdown
AAPL,14.285714,0.215355,1.327641,0.833916,1.345549,-0.090380,-0.385159
C,14.285714,0.196506,1.060960,0.921424,1.288149,-0.154294,-0.565076
AMZN,14.285714,0.328352,1.554363,0.994008,1.358296,-0.114334,-0.533882
DIS,14.285714,0.244820,1.229410,0.954847,1.407096,-0.144185,-0.505998
NVDA,14.285714,0.398036,2.151439,1.300109,1.881499,-0.142095,-0.663351
UNH,14.285714,0.224035,0.389200,0.221559,0.737294,-0.052182,-0.358999
MSFT,14.285714,0.227751,1.380857,0.759801,1.325091,-0.062173,-0.371485


In [11]:
tot_ret = (port_data.iloc[-1] / port_data.iloc[0]) - 1
an_returns = ((1 + tot_ret) ** (1 / 10)) 

In [12]:
result['Ann Tot Ret'] = an_returns

result

,weight,ann_vol,beta_spy,beta_iwm,beta_dia,avg drawdown,max_drawdown,Ann Tot Ret
AAPL,14.285714,0.215355,1.327641,0.833916,1.345549,-0.090380,-0.385159,1.275784
C,14.285714,0.196506,1.060960,0.921424,1.288149,-0.154294,-0.565076,1.002804
AMZN,14.285714,0.328352,1.554363,0.994008,1.358296,-0.114334,-0.533882,1.234079
DIS,14.285714,0.244820,1.229410,0.954847,1.407096,-0.144185,-0.505998,1.030531
NVDA,14.285714,0.398036,2.151439,1.300109,1.881499,-0.142095,-0.663351,1.614606
UNH,14.285714,0.224035,0.389200,0.221559,0.737294,-0.052182,-0.358999,1.234895
MSFT,14.285714,0.227751,1.380857,0.759801,1.325091,-0.062173,-0.371485,1.275952


In [14]:
riskdf = pd.DataFrame(index = etfs)

riskdf['corr'] = p_returns.corrwith(risk_data)

#covar = p_returns.cov(risk_data)
#riskdf['covar'] = covar

#riskdf['track_err'] = np.sqrt(covar.diagonal())

riskdf['sharpe ratio'] = (tot_ret - 0.02) / ann_vol

riskdf


,corr,sharpe ratio
SPY,0.009530,18.219474
IWM,0.018384,5.583015
DIA,0.007047,18.909199


In [ ]:
#risk_vol = port_data.pct_change().std() * np.sqrt(252)
#vol_spr = ann_vol - risk_vol

#riskdf['annual vol spread'] = vol_spr

In [16]:
corr_mat = pd.concat([port_data, risk_data], axis=1).corr()

In [17]:
print("\nPortfolio Risk Analysis:")
print(result)

#print("ETF Portfolio Risk:")
#print(riskdf)

print("\nCorrelation Matrix:")
print(corr_mat)


Portfolio Risk against Benchmarks:
         weight   ann_vol  beta_spy  beta_iwm  beta_dia  avg drawdown  \
AAPL  14.285714  0.215355  1.327641  0.833916  1.345549     -0.090380   
C     14.285714  0.196506  1.060960  0.921424  1.288149     -0.154294   
AMZN  14.285714  0.328352  1.554363  0.994008  1.358296     -0.114334   
DIS   14.285714  0.244820  1.229410  0.954847  1.407096     -0.144185   
NVDA  14.285714  0.398036  2.151439  1.300109  1.881499     -0.142095   
UNH   14.285714  0.224035  0.389200  0.221559  0.737294     -0.052182   
MSFT  14.285714  0.227751  1.380857  0.759801  1.325091     -0.062173   

      max_drawdown  Ann Tot Ret  
AAPL     -0.385159     1.275784  
C        -0.565076     1.002804  
AMZN     -0.533882     1.234079  
DIS      -0.505998     1.030531  
NVDA     -0.663351     1.614606  
UNH      -0.358999     1.234895  
MSFT     -0.371485     1.275952  

Correlation Matrix:
          AAPL      AMZN         C       DIA       DIS       IWM      MSFT  \
AAPL  1.